- Inicializou a população ;
- Quer calcular o fitness dos pais :
  - Indice ibovespa;
  - Media ponderada dos indices de crescimento de cada cromossomo:
    - Pra cada alelo:
      - (Close - Open)/Open ; __só para um dia??__
  - Fitness = min((Média - Indice)^2)
  

- Indice no dia;

-Dataframe =  2 colunas:
 - Primeira: Label;
 - Segunda: Crescimento no dia;
 - Um dia em particular.

In [1]:
import pandas as pd
import datetime
import bovespa as bv
import numpy as np
import pickle
import random
import pygmo
import pandas as pd

In [2]:
ibvday = pd.read_excel("dados/IBOVDIA.XLS", header = 1)
ibvday.set_index("PREGÃO", inplace = True)

In [3]:
ibvday.head()

,JAN,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
PREGÃO,,,,,,,,,,,,
1,NaN,NaN,NaN,9138.3,NaN,NaN,13002.2,12259.1,10109.1,12173.4,NaN,9648.4
2,6955.54,NaN,NaN,9237.3,10074.2,11360.8,13259.6,NaN,11062.9,12396.6,NaN,9964.3
3,6997.75,8095.94,8978.2,9302.1,NaN,11110.4,13284.8,NaN,11119.4,12540.9,9858.0,9991.7
4,NaN,8102.25,8961.4,9705.2,NaN,11029.9,13493.8,12016.9,11430.6,NaN,10254.1,10046.3
5,NaN,8171.18,9074.4,NaN,9993.5,11106.3,NaN,11908.6,11838.3,NaN,9988.3,10080.0


In [4]:
luanindex = ((ibvday.loc[13]["JUN"] - ibvday.loc[12]["JUN"])/ibvday.loc[12]["JUN"]) 

In [5]:
luanindex

-0.00198282117484264

In [6]:
empresas_top = ["CSNA3", "RADL3", "BVMF3", "MULT3",
                "SBSP3", "CPFE3", "BBDC4", "BRML3",
                "BBDC3", "EQTL3", "CSAN3", "PETR4",
                "SMLE3", "LREN3", "KROT3", "PETR3",
                "USIM5", "CMIG4", "QUAL3", "MRVE3",
                "GGBR4", "VALE5", "CCRO3", "VIVT4",
                "VALE3", "GOAU4", "ITSA4", "CIEL3",
                "TBLE3", "EMBR3"]

In [7]:
bf = bv.File("dados/COTAHIST_A2018.TXT")
acc = []
for rec in bf.query():
    if rec.date.year == 2018 and (rec.date.month == 6) :
        acc.append(rec)

In [8]:
subset = dict()

for (key, value) in zip(['date', 'price_open', "price_close", "stock_code"],
                        list(zip(*list(map(lambda foo: (foo.date, 
                                                    foo.price_open,
                                                    foo.price_close,
                                                    foo.stock_code, foo.company_name), acc))))):
    subset[key] = list(value)
    

In [9]:
df = pd.DataFrame.from_dict(subset)
topdf = df[df['stock_code'].isin(empresas_top)]

In [10]:
ourdaydf = (topdf[topdf["date"].map(lambda x: (x.day == 12 or x.day == 13))]
            .set_index("stock_code")
            .drop("price_open", axis = 1))

In [11]:
df13 = ourdaydf[ourdaydf['date'] == datetime.date(2018, 6, 13)]
df12 = ourdaydf[ourdaydf['date'] == datetime.date(2018, 6, 12)]

In [12]:
stock12_13_variation = ((df13["price_close"] - df12["price_close"])/df12["price_close"]).dropna()

In [15]:
# import pickle
# pickle.dump((luanindex,stock12_13_variation ) , open("stockstuff.pickle.dat", "wb"))

In [2]:
import pickle 

luanindex,stock12_13_variation= pickle.load(open("stockstuff.pickle.dat", "rb"))

In [3]:
#criar população 
def inicializacao_populacao():
    pop =  [[]]*50
    pop = [list(zip(np.array(random.sample(range(26), 12)), np.random.dirichlet(np.ones(12),size=1)[0])) for i in range(0, 50)]
    pop = np.array(list(map(list, pop)))
    return pop


#cruzamento
def cruzamento(pai_1, pai_2):
    child = list(zip(pai_1[:,0], pai_2[:,1]))
    return child



In [7]:
faturamento_empresas_top = stock12_13_variation
ibovespa =  luanindex

def function_fitness_risk(pop): 
    return [np.abs(np.sum(faturamento_empresas_top[w[:,0].astype('int')]*w[:,1]) - ibovespa) for w in pop]

def function_fitness_std(pop):
    return(list(map(lambda x: np.std(x[:,1]), pop)))

In [296]:
faturamento_empresas_top = stock12_13_variation
ibovespa =  luanindex
#inicializa a população
pop = inicializacao_populacao()
df = []
df2 = []

pop_fitness_sort2 = []


for j in range(0, 100):
    #calcula o fitness da população
    fitness_risk = function_fitness_risk(pop)
    fitness_std = function_fitness_std(pop)
    
    points = list(zip(fitness_risk, fitness_std))
    
    # Calcula pareto fronts e respectivos crowding_distances
    
    _, _, _, non_domination_rank = pygmo.fast_non_dominated_sorting(points = points)
    crowding_distance = pygmo.crowding_distance(points)
    
    df = pd.DataFrame.from_dict({"popindex": list(range(len(pop))),
                             "ndr": non_domination_rank, "cd": crowding_distance})
    
    pop_fitness_sort = pop[df.sort_values(["ndr", "cd"], 
                                          ascending = [True, False]).popindex.values];
    parent_1 = pop_fitness_sort[0:25]
    parent_2 = pop_fitness_sort[25::]
    #cruzamento
    child_1 = np.array(list(map(list,
                                [cruzamento(parent_1[i], parent_2[i]) for i in range(0, len(parent_1))])))
    child_2 = np.array(list(map(list,
                                [cruzamento(parent_2[i], parent_1[i]) for i in range(0, len(parent_1))])))
    
    #calcula o fitness dos filhos para escolher quem vai passar pra próxima geração
    childs = np.concatenate((child_1, child_2))
    fitness_risk_childs, fitness_std_childs = tuple(map(lambda fn: fn(childs),
                                                        [function_fitness_risk,
                                                         function_fitness_std]))
    
    #seleciona individuos da proxima geração
    fitness_risk.extend(fitness_risk_childs)
    fitness_std.extend(fitness_std_childs)
    points2 = list(zip(fitness_risk,
                      fitness_std))

    _, _, _, non_domination_rank = pygmo.fast_non_dominated_sorting(points = points2)
    crowding_distance = pygmo.crowding_distance(points2)
    
    df2 = pd.DataFrame.from_dict({"popindex": list(range(len(np.concatenate((pop,childs))))),
                             "ndr": non_domination_rank, "cd": crowding_distance})
    
    pop_fitness_sort2 = np.concatenate((pop,childs))[df2.sort_values(["ndr", "cd"], 
                                          ascending = [True, False]).popindex.values];
    
    best_individual = pop_fitness_sort2[0]
    pop = pop_fitness_sort2[0:50]


In [299]:
df2.sort_values(["ndr", "cd"], ascending = [True, False]).head(10)

,popindex,ndr,cd
1,1,0,inf
45,45,0,inf
9,9,0,0.872979
47,47,0,0.131535
43,43,0,0.088105
52,52,0,0.004514
0,0,0,0.001479
44,44,0,0.001479
2,2,0,0.000000
3,3,0,0.000000


In [320]:
list(map(lambda x: list(map(lambda fn: fn([x]),[function_fitness_risk, function_fitness_std])), 
         [pop[i] for i in range(5)]))

[[[9.73923579954735e-06], [0.06958742876701844]],
 [[4.157184289852703e-05], [0.04547620443861979]],
 [[1.7594150354864092e-05], [0.048647671031677296]],
 [[4.157184289852703e-05], [0.04547620443861979]],
 [[4.157184289852703e-05], [0.04547620443861979]]]